In [206]:
import pandas as pd
import numpy as np
import copy

In [207]:
directory = 'categoried/'
files = ['Military Strength','Diplomatic Stance','Conquerability','Diplomatic Opinion',
              'Economy','Prediction','Confidence','Danger','Info Request','Embassy',
              'Trading','War']
file_extension = '.xlsx'



In [266]:
class Parser:
    def __init__(self,directory,filename,extension):
        self.file = directory+filename+extension
        self.cat = filename
        self.df = pd.read_excel(self.file)
        self.initialize_rules(self.cat)
        self.create_parsed()
        
        for i in range(len(self.df.iloc[:])):
            parsed = self.parse_it(self.df.iloc[i])
            self.pardf.append(parsed)
        
    def initialize_rules(self,cat):
        self.category = cat
        self.define_placenames()
        self.author_preps = {'We ':'Author',' we ':'Author','Our':'Author',' our ':'Author',
                             ' ours ':'Author',' us ':'Author'}
        self.subject_preps = {'You ':'Recipient',' you':'Recipient'}
        self.nonspec_words = copy.copy(self.placenames)
        self.nonspec_words.update(self.author_preps)
        self.nonspec_words.update(self.subject_preps)
        
        self.special_rules(cat)
        
    def define_placenames(self):

        self.country_keys = {'The Songhais':'Songhai','the Songhais':'Songhai','Songhai':'Songhai',
                         'The Germans':'Germany','the Germans':'Germany','Germany':'Germany','German':'Germany',
                         'The Koreans':'Korea','the Koreans':'Korea','Korean':'Korea','Korea':'Korea',
                         'The Moroccans':'Morocco','the Moroccans':'Morocco','Moroccan':'Morocco','Morocco':'Morocco',
                         'The Incas':'Inca','the Incas':'Inca','Incan':'Inca','Inca':'Inca',
                         'The Iroquois':'Iroquois','the Iroquois':'Iroquois','Iroquois':'Iroquois'}
        self.countries = {'Songhai':'Songhai','German':'Germany','Morocc':'Morocco','Inca':'Inca',
                          'Korea':'Korea','Iroquoi':'Iroquois'}
        self.city_states = {'Bogota':'Bogota','Budapest':'Budapest','Florence':'Florence','Ife':'Ife',
                            'Kathmandu':'Kathmandu','Kyzyl':'Kyzyl','Mombasa':'Mombasa',
                            'Panama City':'Panama City','Singapore':'Singapore','Yerevan':'Yerevan'}
        self.placenames = copy.copy(self.city_states)
        self.placenames.update(self.countries)

    def special_rules(self,cat):
        if cat == 'Military Strength':
            self.key_dict = {'pathetic':[['pathetic'],[]],'weak':[['weak'],[]],'poor':[['poor'],[]],
                             'average':[['average'],[]],
                             'strong':[['strong'],[]],'powerful':[['powerful'],[]],'immense':[['immense'],[]]}
            self.ignore_refs = ['Recipient']
            self.special_cases = {}
            self.flip_phrases = ['according to']

        if cat == 'Diplomatic Stance':
            self.key_dict = {'neutral':[['neutral'],[]],'hostile':[['hostile'],[]],'afraid':[['afraid'],[]],
                             'aggressive':[['aggressive'],[]],'deceptive':[['deceptive'],[]],
                             'protective':[['protective'],[]],'friendly':[['friend'],[]],
                             'guarded':[['guarded'],[]],'average':[['average'],[]],'FUCKED UP':[['archnemesis','ally'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = ['is what']


        if cat == 'Conquerability':
            self.key_dict = {'impossible':[['impossible'],[]],'soft':[['soft'],[]],' bad':[[' bad'],[]],
                             'favorable':[['favorable'],[]],'average':[['average'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = []

        if cat == 'Diplomatic Opinion':
            self.key_dict = {'ally':[[' ally'],[]],'favorable':[['favorable'],[]],'competitor':[['competitor'],[]],
                             'enemy':[['enemy'],[]],'friend':[['friend'],[]],
                             'archnemesis':[['archnemesis'],[]],'neutral':[['neutral'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = ['Whatever','Inasmuch','Seeing as']
            
        if cat == 'Economy':
            self.key_dict = {'powerful':[[' powerful'],[]],'average':[['average'],[]],'strong':[['strong'],[]],
                             'pathetic':[['pathetic'],[]],'poor':[['poor'],[]],
                             'weak':[['weak'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = ['according']

        if cat == 'Prediction':
            self.key_dict = {'technological':[[' technological'],[]],'cultural':[['cultural'],[]],
                             'military':[['military'],[]],
                             'diplomatic':[['diplomatic'],[]],'unknown':[['unknown'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = []

        if cat == 'Confidence':
            self.key_dict = {'Very sure':[['a very','are very','quite','much'],[]],
                             'Completely sure':[['acutely','thoroughly','most certain','confident'],[]],
                             'Unsure':[['not at all','not very'],[]],
                             'Somewhat sure':[['somewhat'],[]],}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = []

        if cat == 'Danger':
            self.key_dict = {'critical':[[' critical'],[]],'severe':[['severe'],[]],
                             'minor':[['minor'],[]],
                             'moderate':[['moderate'],[]],'major':[['major'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = ['by our','by the']

        if cat == 'Info Request':
            self.key_dict = {'economy':[['economic'],[]],'military':[['military'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = []

        if cat == 'Embassy':
            self.key_dict = {'Embassy establish':[['embassy','seat of '],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = []

        if cat == 'Trading':
            self.key_dict = {'trading':[['We'],[]]}
            self.ignore_refs = []
            self.special_cases = {}
            self.flip_phrases = []

        if cat == 'War':
            self.key_dict = {'War':[['war','conflict','fighting','violence'],
                                    ['terminated','armistice','halted',' end ','stopped','ended','ceased']],
                              'Peace':[['terminated','peace',' end ','ended','armistice',
                                        'halted','stopped','ceased'],[]]}
            self.ignore_refs = ['Recipient']
            self.special_cases = {"['Civ', 'Civ', 'Civ']":[0,2]}
            self.flip_phrases = []
        
        self.key_words = []
        for i in self.key_dict:
            self.key_words.extend(self.key_dict[i][0])
            self.key_words.extend(self.key_dict[i][1])
        a = np.array(self.key_words)
        self.key_words = list(np.unique(a))
            
    def create_parsed(self):
        self.pardf = pd.DataFrame(columns = ['Turn','Civ1','Civ2','Relation','Value','Cable'])
        
    def parse_it(self,linedf):        
        cable = linedf['cable']
        self.turn = linedf['turn']
        self.author = linedf['author civ']
        self.recipient = linedf['subject civ']
        
        descriptors = self.get_descriptors(cable)
        final_descriptor = self.get_key(descriptors)
        
        ordered_refs = self.get_players(cable)
        p1,p2 = self.parse_players(ordered_refs,cable)
    
        parsed = pd.DataFrame([[self.turn,p1,p2,self.cat,final_descriptor,cable]],
                              columns = ['Turn','Civ1','Civ2','Relation','Value','Cable'])
        self.pardf = self.pardf.append(parsed)
        
    def get_descriptors(self,cable):
        cable = cable
        big_words = []
        for i in self.key_words:
            if i in cable:
                big_words.append(i)
        return(big_words)
    
    def get_key(self,descriptors):
        des = descriptors
        final = []
        for i in self.key_dict:
            for j in self.key_dict[i][0]:
                if j in des:
                    final.append(i)
            b = np.array(final)
            final = list(np.unique(b))
            for j in self.key_dict[i][1]:
                if j in des:
                    if i in final:
                        final.remove(i)
        return(final[0])
    
    def get_players(self,cable):
        cable = cable
        referents = {}
        for i in self.nonspec_words:
            if i in cable:
                referents[cable.find(i)] = self.nonspec_words[i]
        ordered_refs = []
        for j in sorted(referents):
            ordered_refs.append(referents[j])          
        return(ordered_refs)
    
    def parse_players(self,ordered_refs,cable):
        cable = cable
        refs = ordered_refs
        refs = self.clean_refs(refs)
        
        return(self.understand_refs(refs,cable))
        

        
    def clean_refs(self,ordered_refs):
        refs = ordered_refs
        for i in self.ignore_refs:
            while i in refs:
                refs.remove(i)
        types = list(np.unique(refs))
        for i in types:
            while refs.count(i) > 1:
                refs.reverse()
                refs.remove(i)
                refs.reverse()
        return(refs)
            
    def understand_refs(self,refs,cable):
        refs = refs
        cable = cable
        p1 = ''
        p2 = ''
        if len(refs) == 0:
            p1 = 'Author'
            p2 = 'Recipient'
        if len(refs) == 1:
            if refs == ['Author']:
                p1 = 'Author'
                p2 = 'Recipient'
            else:
                if self.category in ['Info Request','Trading']:
                    p1 = 'Author'
                    p2 = 'Recipient'
                else:
                    p1 = refs[0]
                    p2 = 'Recipient'
        if len(refs) == 2:
            p1 = refs[0]
            p2 = refs[1]
        if len(refs) > 2:
            p1,p2 = self.resolve_many(refs,cable)
            
        if p1 == 'Author':
            p1 = self.country_keys[self.author]
        if p1 == 'Recipient':
            p1 = self.country_keys[self.recipient]
        if p2 == 'Author':
            p2 = self.country_keys[self.author]
        if p2 == 'Recipient':
            p2 = self.country_keys[self.recipient]
        
        for i in self.flip_phrases:
            if i in cable:
                return(p2,p1)
        
        return(p1,p2)
    
    def resolve_many(self,refs,cable):
        refs = refs
        cable = cable
        ref_types = []
        for i in refs:
            ref_types.append(self.reftype(i))
        key = str(ref_types)
        if key in self.special_cases:
            p1 = refs[self.special_cases[key][0]]
            p2 = refs[self.special_cases[key][1]]
        else:
            p1 = refs[0]
            p2 = refs[1]
        return(p1,p2)
            
    def reftype(self,ref):
        civ = ['Author','Recipient']
        civ.extend(self.countries.values())
        city = self.city_states.values()
        if ref in city:
            return('City')
        if ref in civ:
            return('Civ')
        print('Error: reftype key failure')
        
        
for i in files:
    d = Parser(directory,i,file_extension)
    d.pardf.to_csv('Finished/'+i+'.csv')
    print('saved',i)
   
print('done')     
        

done


In [264]:
for i in range(0,len(d.pardf)):
    print(d.pardf.iloc[i].Turn,'||',d.pardf.iloc[i].Civ1,'||',d.pardf.iloc[i].Civ2,'||',d.pardf.iloc[i].Value)
    print(d.pardf.iloc[i].Cable)
    print()

59 || Songhai || Inca || trading
We needed an accord to grant a diplomatic outpost in The Incass capital and thus proposed 2 units of gold per turn for the next 30 turns in return

72 || Korea || Inca || trading
We desired 1 unit of silver for the upcoming 30 turns and thus extended 1 unit of copper for the subsequent 30 turns in compensation

72 || Morocco || Iroquois || trading
We were enthusiastic to bargain with The Iroquois mostly for 2 units of gold per turn for the coming 30 turns

83 || Germany || Korea || trading
We wished for 1 unit of copper for the subsequent 30 turns and thus gave 1 unit of salt for the upcoming 30 turns in compensation

84 || Morocco || Inca || trading
We desired 1 unit of silver for the next 30 turns and thus gave 1 unit of cotton for the subsequent 30 turns and 10 units of gold per turn for the subsequent 30 turns in exchange

102 || Germany || Iroquois || trading
We were giddy to deal with The Iroquois mostly for 1 unit of spices for the subsequent 30 

In [47]:
a = 'abcdefghijklmnopqrstuvwxyz'
s = ['def','mn','bcde','abc','bobby']
for i in s:
    print(a.find(i))

3
12
1
0
-1
